In [1]:
import joblib
model_data = joblib.load("/content/rf_pipeline.pkl")

In [2]:
#Extracting Components
pipeline = model_data["pipeline"]
label_encoder = model_data["label_encoder"]
feature_cols = model_data["feature_cols"]
budget_col = model_data["budget_col"]
product_price_col = model_data["product_price_col"]
expense_cols = model_data["expense_cols"]
month_col = model_data["month_col"]

Prediction

In [3]:
import pandas as pd

def predict_from_pkl(month, budget, expenses_dict, product_name, product_price, top_n_suggestions=5):
    # Build input row
    row = {}
    row[budget_col] = budget
    for c in expense_cols:
        row[c] = float(expenses_dict.get(c, 0))

    # Compute initial total
    row["initial_total"] = sum(float(expenses_dict.get(c, 0)) for c in expense_cols)

    # Add product price + month
    row[product_price_col] = product_price
    row[month_col] = month

    # Ensure correct feature order
    input_df = pd.DataFrame([row], columns=feature_cols)

    # Make prediction
    pred_enc = pipeline.predict(input_df)[0]
    pred_label = label_encoder.inverse_transform([pred_enc])[0]

    # Balance calculation
    balance = budget - row["initial_total"]

    # Suggestions if product price > balance
    suggestions = []
    if product_price > balance:
        pred_label = "No"
        cand = df[df[product_price_col] <= balance]
        if not cand.empty:
            suggestions = cand.sort_values(by=product_price_col, ascending=False).head(top_n_suggestions)

    return {
        "initial_total": row["initial_total"],
        "balance": balance,
        "prediction": str(pred_label),
        "suggestions": suggestions
    }


User Input

In [4]:
month = input("Enter Month (e.g., December): ")
budget = float(input("Enter total budget: "))

expenses_dict = {}
for col in expense_cols:
    val = input(f"Enter expense for {col} (leave blank for 0): ")
    expenses_dict[col] = float(val) if val.strip() != "" else 0.0

product_name = input("Enter product name: ")
product_price = float(input("Enter product price: "))

result = predict_from_pkl(month, budget, expenses_dict, product_name, product_price)

print("\n===== RESULT =====")
print(f"Initial Expense: {result['initial_total']}")
print(f"Remaining Balance: {result['balance']}")
print(f"Prediction (Can Buy?): {result['prediction']}")

if result["prediction"] == "No":
    print("\nSuggested Products within Balance:")
    if not result["suggestions"].empty:
        print(result["suggestions"][[product_price_col, month_col]].to_string(index=False))
    else:
        print("No products found within balance.")


Enter Month (e.g., December): February
Enter total budget: 21000
Enter expense for Grocery_Ini1 (leave blank for 0): 2500
Enter expense for Rent_Ini2 (leave blank for 0): 2350
Enter expense for Transport_Ini3 (leave blank for 0): 1150
Enter product name: Dress
Enter product price: 6000

===== RESULT =====
Initial Expense: 6000.0
Remaining Balance: 15000.0
Prediction (Can Buy?): Yes, But check 'Have Balance'.
